In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,time,re,json
from glob import glob
import datetime
import sqlite3

from PIL import Image, ImageDraw
import cv2

## Define Path

In [ ]:
PATH=os.path.join('..','data','phase-01','data')

## Define Tile Class

In [ ]:
class TILE:
    def __init__(self, filepath):
        img = cv2.imread(filepath)
        self.array = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.shape = self.array.shape
        msk_img = cv2.imread(re.findall('^.*sentinel.*?/', file)[0]+'masks/sugarcane-region-mask.png')
        self.mask = cv2.cvtColor(msk_img, cv2.COLOR_BGR2RGB)
    def image(self):
        plt.figure(figsize=(8,8))
        self.image = plt.imshow(self.array)
    def detect_harvested(self):
        setTime = time.time()
        plt.figure(figsize=(8,8))
        pixels = self.array.copy()
        mask = self.mask
        x = pixels.reshape(512**2, 3)
        x = pd.DataFrame(x, columns=['r','g','b'])
        mask = pd.DataFrame(mask.reshape(512**2, 3), index = x.index)
        mask_idx = mask[~(mask.sum(axis=1)==0)].index
        x.loc[mask_idx, 'r'] = 0
        x.loc[mask_idx, 'g'] = 0
        x.loc[mask_idx, 'b'] = 0
        idx = x[x.g/(x.sum(axis=1)) < 0.335].index.difference(mask_idx)
        x.loc[idx, 'r'] = 255
        x.loc[idx, 'g'] = 0
        x.loc[idx, 'b'] = 0
        x = x.values.reshape(512,512,3)
        self.area = 100*(len(idx)/1000)
        pixels = np.array([0,0,255]*512**2).reshape(512,512,3)
        return plt.imshow(x)
    def harvested_area(self):
        if 'self.area' not in locals():
            pixels = self.array.copy()
            mask = self.mask
            width, height, _ = pixels.shape
            x = pixels.reshape(512**2, 3)
            x = pd.DataFrame(x, columns=['r','g','b'])
            mask = pd.DataFrame(mask.reshape(512**2, 3), index = x.index)
            mask_idx = mask[~(mask.sum(axis=1)==0)].index
            idx = x[x.g/(x.sum(axis=1)) < 0.335].index
            self.area = 100*(len(idx.difference(mask_idx))/1000)
        return self.area

## -----------------------------------------------

### 1) Process timeseries

In [ ]:
def processTimestamp(root,source,files):
    #Iterate all image files
    data=dict()
    cols=['tile_x','tile_y','imgtype','year','month','day']
    for file in files:
        if not '.png' in file: continue
        #Split timestamp
        name=file.rsplit('.',1)[0]
        data[name]=name.split('-')
        
    #Create dataframe    
    df=pd.DataFrame().from_dict(data,orient='index',columns=cols)
    df=df.reset_index().rename({'index':'File'},axis=1)
    
    #Add date
    df['date']=df.apply(lambda x: datetime.date(int(x['year']), int(x['month']), int(x['day'])), axis=1)
    
    return df
     
def processTimeseriesByDate(root,source,files):
    
    cnt=0
    
    #Timestamp process
    df_timestamps=processTimestamp(root,source,files)
    
    #Iterate all dates
    for date in df_timestamps['date'].unique():
        df=pd.DataFrame()
        t0=time.time()
        imgfiles=df_timestamps.loc[df_timestamps['date']==date,['File','imgtype']].values
        print(time.time()-t0)
        for imgfile in imgfiles:
            print('>>>>>')
            #Extract pixel data
            rgb=cv2.cvtColor(cv2.imread(os.path.join(root,imgfile[0]+'.png')), cv2.COLOR_BGR2RGB)
            t0=time.time()
            
            print('------')
            #Create frame
            colIndex=pd.MultiIndex.from_arrays([[imgfile[1],imgfile[1],imgfile[1]], ['r','g','b']], names=('imgtype', 'rgb'))
            data=pd.DataFrame(rgb.reshape(len(rgb[0])**2, 3),columns=colIndex)
            print(time.time()-t0)
            t0=time.time()

            #Append data
            df=data if df.empty else df.merge(data,left_index=True,right_index=True)
            print(time.time()-t0)
            t0=time.time()
        
        #Index
        df.index.name='Pixel'
        #temp.index=pd.MultiIndex.from_arrays([[imgfile[0] for _ in range(len(temp))], [i for i in range(len(temp))]], names=('File', 'Pixel'))
        #df=temp if df.empty else df.append(temp)
        
        print(time.time()-t0)
        t0=time.time()
        #Output image data
        imgpath=os.path.join(parsedpath,'timeseries')
        if not os.path.exists(imgpath):
            os.makedirs(imgpath)
        df.to_csv(os.path.join(imgpath,date.strftime("%Y-%m-%d")+'.csv'))
             
        print(time.time()-t0)
        
        if cnt%20==0:
            print(cnt)
            break
        cnt+=1
        
    return None


def processTimeseriesByPixel(root,source,files):
    
    cnt=0
    
    #Timestamp process
    df_timestamps=processTimestamp(root,source,files)

    ts=time.time()
    #Iterate all dates
    df_full=pd.DataFrame()
    for date in df_timestamps['date'].unique():
        df=pd.DataFrame()
        imgfiles=df_timestamps.loc[df_timestamps['date']==date,['File','imgtype']].values
        for imgfile in imgfiles:
            
            #Extract pixel data
            rgb=cv2.cvtColor(cv2.imread(os.path.join(root,imgfile[0]+'.png')), cv2.COLOR_BGR2RGB)
            
            #Create frame
            colIndex=pd.MultiIndex.from_arrays([[imgfile[1],imgfile[1],imgfile[1]], ['r','g','b']], names=('imgtype', 'rgb'))
            data=pd.DataFrame(rgb.reshape(len(rgb[0])**2, 3),columns=colIndex)
            
            #Append data
            df=data if df.empty else df.merge(data,left_index=True,right_index=True)
        
        #Add in more features!!!!
        #df['Harvested']=df.apply(lambda x: getHarvestState(x), axis=1)
        
        #Set indexes
        df.index.name='Pixel'
        df['Date'] = date
        df.set_index('Date', append=True, inplace=True)
        
        #Append data
        df_full=df if df_full.empty else df_full.append(df)
        print(df_full.shape)
        
        if cnt%20==0:
            print(cnt)
            #break
        cnt+=1
        
    print(time.time()-ts)

    if(0):
        t0=time.time()
        #Output image data
        imgpath=os.path.join(parsedpath,'timeseries')
        if not os.path.exists(imgpath):
            os.makedirs(imgpath)
        df_full.to_hdf(os.path.join(imgpath,'data.csv'), 'data', mode='w')
        #df_full.to_csv(os.path.join(imgpath,date.strftime("%Y-%m-%d")+'.csv'))
        print(time.time()-t0)
        
    return df_full

def createSQL(df):
    
    t0=time.time()
    #Output image data
    table_name='images'
    dbpath=os.path.join(parsedpath,'timeseries.db')
    if not os.path.exists(dbpath):
        open(dbpath,'w').close()
    conn = sqlite3.connect(dbpath)
    c = conn.cursor()

    #Create sql
    sql='DROP TABLE IF EXISTS '+table_name
    c.execute(sql)
    conn.commit()
    
    #Columns in sql
    sqlCols=''
    for col in df:
        dtype='INTEGER' if 'int' in str(df[col].dtypes) else 'CHAR(32)'
        sqlCols+=col+' '+dtype+','
    sql='CREATE TABLE '+table_name+' ('+sqlCols[:-1]+');'
    c.execute(sql)
    conn.commit()

    #Insert sql
    inserts=[]
    for i in range(len(df)):
        sql='INSERT INTO '+table_name+' (' + ','.join([col for col in df]) + ') '
        sql+='VALUES(' + ','.join(['?' for _ in df]) + ')'
        inserts.append(tuple(val for val in list(df.iloc[i].values)))
        
    #Execute statement
    c.executemany(sql,inserts)
    conn.commit()
    conn.close()

    print(time.time()-t0)
    
def processTimeseriesByPixel2(root,source,files):
    
    cnt=0
    
    #Timestamp process
    df_timestamps=processTimestamp(root,source,files)

    ts=time.time()
    #Iterate all dates
    df_full=pd.DataFrame()
    for date in df_timestamps['date'].unique():
        df=pd.DataFrame()
        imgfiles=df_timestamps.loc[df_timestamps['date']==date,['File','imgtype']].values
        for imgfile in imgfiles:
            
            #Extract pixel data
            rgb=cv2.cvtColor(cv2.imread(os.path.join(root,imgfile[0]+'.png')), cv2.COLOR_BGR2RGB)
            
            #Create frame
            data=pd.DataFrame(rgb.reshape(len(rgb[0])**2, 3),columns=['r_'+imgfile[1],'g_'+imgfile[1],'b_'+imgfile[1]])
            
            #Append data
            df=data if df.empty else df.merge(data,left_index=True,right_index=True)
        
        #Set indexes
        df=df.reset_index().rename({'index':'Pixel'},axis=1)
        df['Date'] = date
        
        #Append data
        df_full=df if df_full.empty else df_full.append(df)
        print(df_full.shape)
        
        if cnt%20==5:
            print(cnt)
            break
        cnt+=1
        
    print(time.time()-ts)
    createSQL(df_full)
        
    return df_full

### 2) Process geometry

In [ ]:
def processGeometry(root,source,files):
    
    #Iterate all geojson files
    df=pd.DataFrame()
    data=dict()
    for file in files:
        if not '.geojson' in file: continue
        #Split timestamp
        name=file.rsplit('.',1)[0]
        tile=[''.join(re.findall(r'\d+', s)) for s in name.split('-')[1:]]
        data[name]=dict()
        data[name]['tile_x']=tile[0]
        data[name]['tile_y']=tile[1]
        
        #Read geojson
        with open(os.path.join(root,file),'r') as f:
            geo=json.load(f)
        
        #Extract crs
        data[name]['crs']=geo['crs']['properties']['name']
        
        #Extract coordinates
        for i,coord in enumerate(geo['features'][0]['geometry']['coordinates'][0]):
            data[name]['lat'+str(i)]=coord[1]
            data[name]['lon'+str(i)]=coord[0]
    
        #Append data
        data=pd.DataFrame().from_dict(data,orient='index')
        df=data if df.empty else df.append(data)
                    
    return df.reset_index().rename({'index':'Geojson'},axis=1)

### 3) Process masks

In [ ]:
def processMasks(root,source,files):
    
    #Iterate all masks
    cols=['r','g','b']
    df=pd.DataFrame()
    for file in files:
        if not '.png' in file: continue
        
        #Extract pixel data
        rgb=cv2.cvtColor(cv2.imread(os.path.join(root,file)), cv2.COLOR_BGR2RGB)
        
        #Create frame
        data=pd.DataFrame(rgb.reshape(len(rgb[0])**2, 3),columns=cols)
        data['Mask']=file.rsplit('.',1)[0]
        
        #Append data
        df=data if df.empty else df.append(data)
        
    return df


### 4) Process metadata

In [ ]:
def processMetadata(root,source,files):
    
    #Iterate all metadata
    df=pd.DataFrame()
    cols=['year','month','day']
    for file in files:
        data=dict()
        if not '.json' in file: continue
        #Split timestamp
        name=file.rsplit('.',1)[0]
        date=name.split('-')
        data[name]=dict()
        data[name]['year']=date[0]
        data[name]['month']=date[1]
        data[name]['day']=date[2]
        
        #Read geojson
        with open(os.path.join(root,file),'r') as f:
            metadata=json.load(f)['metadata']
        
        #Metrics
        inds=['collection','processingLevel','platform','instrument','resolution','sensorMode','orbitNumber','snowCover','cloudCover']
        for ind in inds:
            data[name][ind]=metadata[ind]
        
        #Add centroid
        data[name]['lat_centroid']=metadata['centroid']['coordinates'][1]
        data[name]['lon_centroid']=metadata['centroid']['coordinates'][0]
        
        #Append data
        data=pd.DataFrame().from_dict(data,orient='index')
        df=data if df.empty else df.append(data)
                    
    return df.reset_index().rename({'index':'Metadata'},axis=1)

### Iterate through files

In [ ]:
#Walk path
#startdate=datetime.date(2016, 1, 1)
#enddate=datetime.date(2017, 1, 1)
for root, dirs, files in os.walk(PATH,topdown=True):
    
    #Parsed directory
    tile=os.path.split(os.path.split(root)[0])[1]
    if 'tile' not in tile: continue
    parsedpath=os.path.join(os.path.split(PATH)[0],'parsed',tile)
    if not os.path.exists(parsedpath):
        os.makedirs(parsedpath)
        
    #Source types
    source=os.path.split(root)[-1]
    if source=='timeseries':
        #df=processTimeseriesByDate(root,source,files)
        df=processTimeseriesByPixel2(root,source,files)
    
    if source=='geometry': 
        df=processGeometry(root,source,files)
        df.to_csv(os.path.join(parsedpath,'geometry.csv'))

    if source=='masks': 
        df=processMasks(root,source,files)
        df.to_csv(os.path.join(parsedpath,'masks.csv'))

    if source=='metadata':
        df=processMetadata(root,source,files)
        df.to_csv(os.path.join(parsedpath,'metadata.csv'))

print(df.shape)
df.head()

## Things to do

In [ ]:
#multi process 

#Image visualize function with layers

#harvest/unharvested by pixel
#size of harvest area attached to pixel

#Neighbour cells activity (harvest/not harvest)
#Neighbour avg rgb (convolution kernel)
#Neighbour cells landforms (rivers,trees,hills,etc)

#Distance to landforms,water sources, etc.

#Weather,temp,sun,radiation on day?
#Soil info?

